In [408]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import TargetEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor


In [409]:
data_path = "./data/lot41_statistical.csv"
df = pd.read_csv(data_path)
df.head(5)

,name,price,brand,item_condition,shipping,located_in,return_policy,category,money_back,seller_item_sold,trending,price_log,price_boxcox
0,Mens Jogging Bottoms Elasticated Trouser Jogge...,14.48,Crazy Girl Ltd,New with tags,(approx C $10.20),United Kingdom,14 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Clothi...",Yes,203K,Yes/3,2.739549,3.488990
1,Men's 100% Cotton Big Polka Dot Design Spread...,29.19,George/Fortino Landi,New with tags,(approx C $7.30),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Clothi...",Yes,49K,Yes/56,3.407511,4.739931
2,G-Unit Style heavy weigh Tank Top Square Cut ...,15.89,Basix,New with tags,(approx C $25.88),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Clothi...",Yes,207K,Yes/5,2.826722,3.645312
3,Men's Fashion Oxford Faux Leather Dress Shoes...,33.23,Milano Moda & Alberto Fellini,New with box,(approx C $56.04),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Shoes/...",Yes,49K,Yes/715,3.533102,4.990177
4,Women's Cute Caged Peep Toe Low High Platform ...,15.81,Top Moda,New without box,(approx C $30.72),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Women/Women's Sh...",Yes,326K,No,2.821974,3.636750


In [410]:
df.shape

(17893, 13)

In [411]:
tfid = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfid.fit_transform(df['name'])
k_means = KMeans(n_clusters=50, random_state=0)
k_means.fit(tfidf_matrix)
df['name'] = k_means.labels_
df.head()

/Users/jaxxsulav/Projects/Loyalist/Sem2/Step/PriceProphet/env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,name,price,brand,item_condition,shipping,located_in,return_policy,category,money_back,seller_item_sold,trending,price_log,price_boxcox
0,43,14.48,Crazy Girl Ltd,New with tags,(approx C $10.20),United Kingdom,14 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Clothi...",Yes,203K,Yes/3,2.739549,3.488990
1,2,29.19,George/Fortino Landi,New with tags,(approx C $7.30),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Clothi...",Yes,49K,Yes/56,3.407511,4.739931
2,3,15.89,Basix,New with tags,(approx C $25.88),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Clothi...",Yes,207K,Yes/5,2.826722,3.645312
3,1,33.23,Milano Moda & Alberto Fellini,New with box,(approx C $56.04),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Men/Men's Shoes/...",Yes,49K,Yes/715,3.533102,4.990177
4,49,15.81,Top Moda,New without box,(approx C $30.72),United States,30 days return . Buyer pays for return shippi...,"Clothing, Shoes & Accessories/Women/Women's Sh...",Yes,326K,No,2.821974,3.636750


##### Binarizing return_policy, money_back, trending

In [412]:
df['return_policy'] = df['return_policy'].apply(lambda x: 0 if 'no' in x.lower() else 1)
df['money_back'] = df['money_back'].apply(lambda x: 1 if 'yes' in x.lower() else 0)
df['trending'] = df['trending'].apply(lambda x: 1 if 'yes' in x.lower() else 0)
df.head()

,name,price,brand,item_condition,shipping,located_in,return_policy,category,money_back,seller_item_sold,trending,price_log,price_boxcox
0,43,14.48,Crazy Girl Ltd,New with tags,(approx C $10.20),United Kingdom,1,"Clothing, Shoes & Accessories/Men/Men's Clothi...",1,203K,1,2.739549,3.488990
1,2,29.19,George/Fortino Landi,New with tags,(approx C $7.30),United States,1,"Clothing, Shoes & Accessories/Men/Men's Clothi...",1,49K,1,3.407511,4.739931
2,3,15.89,Basix,New with tags,(approx C $25.88),United States,1,"Clothing, Shoes & Accessories/Men/Men's Clothi...",1,207K,1,2.826722,3.645312
3,1,33.23,Milano Moda & Alberto Fellini,New with box,(approx C $56.04),United States,1,"Clothing, Shoes & Accessories/Men/Men's Shoes/...",1,49K,1,3.533102,4.990177
4,49,15.81,Top Moda,New without box,(approx C $30.72),United States,1,"Clothing, Shoes & Accessories/Women/Women's Sh...",1,326K,0,2.821974,3.636750


#### Dealing with Category

In [413]:
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

df['category'] = enc.fit_transform(df[['category']]).astype(int)
kv_pair_ic = {category: i for i, category in enumerate(enc.categories_[0])}
print(kv_pair_ic)
print(len(enc.categories_[0]))
df.head()

{"Business & Industrial/Facility Maintenance & Safety/Personal Protective Equipment (PPE)/Protective Jackets/Clothing, Shoes & Accessories/Men/Men's Clothing/Men/Men's Clothing/Activewear/Hoodies & Sweatshirts": 0, "Business & Industrial/Facility Maintenance & Safety/Personal Protective Equipment (PPE)/Protective Jackets/Clothing, Shoes & Accessories/Men/Men's Clothing/Men/Men's Clothing/Shirts/Casual Button-Down Shirts": 1, "Business & Industrial/Facility Maintenance & Safety/Personal Protective Equipment (PPE)/Protective Pants/Clothing, Shoes & Accessories/Men/Men's Clothing/Pants": 2, "Business & Industrial/Facility Maintenance & Safety/Personal Protective Equipment (PPE)/Protective Suits & Coveralls/Clothing, Shoes & Accessories/Men/Men's Clothing/Pants": 3, 'Business & Industrial/Facility Maintenance & Safety/Personal Protective Equipment (PPE)/Work Boots & Shoes': 4, "Business & Industrial/Facility Maintenance & Safety/Personal Protective Equipment (PPE)/Work Boots & Shoes/Clothi

,name,price,brand,item_condition,shipping,located_in,return_policy,category,money_back,seller_item_sold,trending,price_log,price_boxcox
0,43,14.48,Crazy Girl Ltd,New with tags,(approx C $10.20),United Kingdom,1,23,1,203K,1,2.739549,3.488990
1,2,29.19,George/Fortino Landi,New with tags,(approx C $7.30),United States,1,63,1,49K,1,3.407511,4.739931
2,3,15.89,Basix,New with tags,(approx C $25.88),United States,1,74,1,207K,1,2.826722,3.645312
3,1,33.23,Milano Moda & Alberto Fellini,New with box,(approx C $56.04),United States,1,167,1,49K,1,3.533102,4.990177
4,49,15.81,Top Moda,New without box,(approx C $30.72),United States,1,339,1,326K,0,2.821974,3.636750


#### Binarizing the shipping values

In [414]:
df.loc[~df['shipping'].astype(str).str.contains('\$'), 'shipping'] = 0
df.loc[df['shipping'].astype(str).str.contains('\$'), 'shipping'] = 1
df['shipping'] = df['shipping'].astype(float)
df.head()

,name,price,brand,item_condition,shipping,located_in,return_policy,category,money_back,seller_item_sold,trending,price_log,price_boxcox
0,43,14.48,Crazy Girl Ltd,New with tags,1.0,United Kingdom,1,23,1,203K,1,2.739549,3.488990
1,2,29.19,George/Fortino Landi,New with tags,1.0,United States,1,63,1,49K,1,3.407511,4.739931
2,3,15.89,Basix,New with tags,1.0,United States,1,74,1,207K,1,2.826722,3.645312
3,1,33.23,Milano Moda & Alberto Fellini,New with box,1.0,United States,1,167,1,49K,1,3.533102,4.990177
4,49,15.81,Top Moda,New without box,1.0,United States,1,339,1,326K,0,2.821974,3.636750


### Dealing with seller_item_sold

In [415]:
# For "seller_positive_feedback"
def to_numbers(short):
    if 'K' in short:
        return float(short.replace('K', '')) * 1000
    elif 'M' in short:
        return float(short.replace('M', '')) * 1000000
    else:
        return float(short)

df["seller_item_sold"] = df["seller_item_sold"].apply(to_numbers)

### Target Encoder for item_condition

In [416]:
encoder = TargetEncoder()

encoder.fit(df['item_condition'], df['price_log'])
df['item_condition'] = encoder.transform(df['item_condition'])

encoder.fit(df['brand'], df['price_log'])
df['brand'] = encoder.transform(df['brand'])

encoder.fit(df['located_in'], df['price_log'])
df['located_in'] = encoder.transform(df['located_in'])

df.head()

,name,price,brand,item_condition,shipping,located_in,return_policy,category,money_back,seller_item_sold,trending,price_log,price_boxcox
0,43,14.48,3.360062,3.346548,1.0,3.272318,1,23,1,203000.0,1,2.739549,3.488990
1,2,29.19,3.770229,3.346548,1.0,3.893614,1,63,1,49000.0,1,3.407511,4.739931
2,3,15.89,3.694664,3.346548,1.0,3.893614,1,74,1,207000.0,1,2.826722,3.645312
3,1,33.23,3.786570,4.136576,1.0,3.893614,1,167,1,49000.0,1,3.533102,4.990177
4,49,15.81,3.750213,3.890525,1.0,3.893614,1,339,1,326000.0,0,2.821974,3.636750


In [417]:
df.dtypes

name                  int32
price               float64
brand               float64
item_condition      float64
shipping            float64
located_in          float64
return_policy         int64
category              int64
money_back            int64
seller_item_sold    float64
trending              int64
price_log           float64
price_boxcox        float64
dtype: object

In [418]:
df.to_csv("./data/lot42_vectorized.csv", index=False)